In [ ]:
from sklearn import datasets
import numpy as np

In [626]:
iris = datasets.load_iris()

In [ ]:
data = np.column_stack((iris['target'], iris['data']))  # combine target with data, make target the first column like it was in MNIST 

In [630]:
def normalize(X):
    # normalize by feature - each column separately
    return (X - X.min(axis=1, keepdims=True)) / (X.max(axis=1, keepdims=True) - X.min(axis=1, keepdims=True))


In [644]:
data[:5]

array([[0. , 4.9, 3.1, 1.5, 0.1],
       [2. , 6.3, 2.7, 4.9, 1.8],
       [1. , 6.1, 2.8, 4.7, 1.2],
       [0. , 5. , 3.3, 1.4, 0.2],
       [2. , 6.2, 2.8, 4.8, 1.8]])

In [ ]:
m, n = data.shape # rows (150, number of flowers) and columns (number of features + 1 for the label)

# split up data into training and testing
np.random.shuffle(data)

data_dev = data[0:30].T  # 30 flowers for testing
Y_dev = data_dev[0].astype(int)
X_dev = data_dev[1:n]
X_dev =  normalize(X_dev)  # between 0 and 1. Normalize each feature separately according to its min and max

data_train = data[30:m].T
Y_train = data_train[0].astype(int)  # because of transposition, labels are now in the first row
X_train = data_train[1:n]
X_train = normalize(X_train)

In [633]:
Y_train

array([1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2,
       0, 0, 2, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1,
       2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 1,
       1, 0, 1, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 0, 1, 1, 2,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 1])

In [634]:
# Better initial parameters can lead to higher accuracy
def init_parameters():
    W1 = np.random.rand(3, 4) - 0.5  # between -0.5 and 0.5
    b1 = np.random.rand(3, 1) - 0.5
    W2 = np.random.rand(3, 3) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

In [635]:
# Activation functions
def ReLU(Z):  # for 1st layer
    return np.maximum(0, Z) # element-wise operation on Z

def softmax(Z):  # for 2nd layer
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return e_Z / e_Z.sum(axis=0)

def deriv_ReLU(Z):
    return Z > 0  # if Z is positive, returns 1, 0 otherwise

In [636]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1  # since W1 and x are 2D arrays, dot is matrix multiplication
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2

In [ ]:
# Y is 0-2, so max+1 is 3 - how many classes we need
# for each row, go to the column specified by Y (0-2) and set it to 1
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max()+1))  # m rows, 3 columns
    one_hot_Y[np.arange(Y.size), Y] = 1  # pair each row number 0 to m with the corresponding label
    return one_hot_Y.T  # each column is an example

In [638]:
# m is size of Y - how many examples there are
def back_prop(Z1, A1, A2, W2, X, one_hot_Y, m):
    dZ2 = A2 - one_hot_Y  # accounts for derivative of softmax
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2

In [639]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha*dW1
    b1 = b1 - alpha*db1

    W2 = W2 - alpha*dW2
    b2 = b2 - alpha*db2

    return W1, b1, W2, b2


In [640]:
def get_predictions(A2):
    return np.argmax(A2, 0)  # chooses the digit with the highest probability

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [641]:
# alpha is a hyperparameter set by the programmer
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_parameters()
    one_hot_Y = one_hot(Y)
    m = Y.size
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, A2, W2, X, one_hot_Y, m)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i%100 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
        
    return W1, b1, W2, b2

In [ ]:
# train the model
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 1001, 0.1)

Iteration:  0
Accuracy:  0.325
Iteration:  100
Accuracy:  0.675
Iteration:  200
Accuracy:  0.8583333333333333
Iteration:  300
Accuracy:  0.9333333333333333
Iteration:  400
Accuracy:  0.9666666666666667
Iteration:  500
Accuracy:  0.9666666666666667
Iteration:  600
Accuracy:  0.9583333333333334
Iteration:  700
Accuracy:  0.9583333333333334
Iteration:  800
Accuracy:  0.9583333333333334
Iteration:  900
Accuracy:  0.9666666666666667


In [ ]:
# check performance on test dataset
def predict(X, Y, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    acc = get_accuracy(predictions, Y)
    print("Dev Set Accuracy:", acc)
    return predictions

predict(X_dev, Y_dev, W1, b1, W2, b2)

Dev Set Accuracy: 0.9333333333333333


array([0, 2, 1, 0, 2, 1, 1, 2, 2, 1, 1, 0, 0, 1, 1, 2, 2, 2, 0, 1, 0, 1,
       2, 0, 0, 0, 1, 0, 1, 0], dtype=int64)